<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [2]</a>'.</span>

In [1]:
import pandas as pd
import sweetviz as sv
import os

from tqdm import tqdm
from IPython.display import display

print("Librerías importadas correctamente.")

/home/jules/.pyenv/versions/3.12.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Librerías importadas correctamente.


/home/jules/.pyenv/versions/3.12.12/lib/python3.12/site-packages/sweetviz/graph.py:8: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_filename


<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [2]:
import osimport pandas as pdfrom IPython.display import displayruta_datos_crudos = os.path.join('..', 'data', '01_raw', 'dummy_data.csv')df_raw = pd.read_csv(ruta_datos_crudos)print(f"Dataset (dummy) cargado exitosamente. Dimensiones: {df_raw.shape}")display(df_raw.head())

SyntaxError: invalid syntax (1752707783.py, line 1)

In [ ]:
import numpy as np
import warnings

# --- INICIO DEL PARCHE ---
# Revisa si el atributo 'VisibleDeprecationWarning' NO existe en numpy
if not hasattr(np, 'VisibleDeprecationWarning'):
    # Si no existe, crea una clase de advertencia falsa para que actúe como reemplazo
    class VisibleDeprecationWarning(UserWarning):
        pass
    # Asigna la advertencia falsa al módulo numpy
    np.VisibleDeprecationWarning = VisibleDeprecationWarning
# --- FIN DEL PARCHE ---


# Ahora puedes importar y usar sweetviz con normalidad
import sweetviz as sv
import pandas as pd

# (Aquí iría el resto de tu código para cargar el DataFrame y usar sweetviz)
# df_analisis = ...
# reporte = sv.analyze(df_analisis)
# reporte.show_html()

print("Parche aplicado. Sweetviz ahora debería funcionar con NumPy 2.x.")

In [ ]:
# --- CELDA PARA ANÁLISIS EXPLORATORIO (VERSIÓN FINAL) ---

import pandas as pd
import sweetviz as sv
import os

# (Asumimos que df_raw ya está cargado de la celda anterior)

# 1. Define la lista de columnas ampliada y validada.
columnas_enriquecidas = [
    # --- Variables Target (Objetivo) ---
    'CVDINFR4',  # Ataque al corazón (Infarto de Miocardio)
    'CVDCRHD4',  # Angina o Enfermedad Coronaria
    'CVDSTRK3',  # ACV (Stroke)

    # --- Factores de Riesgo Principales ---
    '_AGEG5YR',  # Edad en categorías de 5 años
    'SEXVAR',    # Sexo
    '_BMI5',     # Índice de Masa Corporal (calculado)
    '_SMOKER3',  # Fumador (calculado)
    
    # --- Estilo de Vida y Salud General ---
    'GENHLTH',   # Salud General (percepción)
    'PHYSHLTH',  # Días con mala salud física
    'MENTHLTH',  # Días con mala salud mental
    '_TOTINDA',  # Actividad Física
    'SLEPTIM1',  # Horas de sueño
    '_RFBING6',  # Consumo excesivo de alcohol (Binge Drinking)

    # --- Comorbilidades Relevantes ---
    'DIABETE4',  # Diabetes
    'HAVARTH4',  # Artritis
    'ADDEPEV3',  # Depresión
    'CHCKDNY2',  # Enfermedad Renal Crónica
    
    # --- VARIABLES RECOMENDADAS (Determinantes Sociales) ---
    '_EDUCAG',   # Nivel Educativo
    'MARITAL',   # Estado Civil
    '_INCOMG1',  # Nivel de Ingresos
    '_PRACE2'    # Raza/Etnia
]

# 2. Crea el subconjunto del DataFrame.
print("Creando un subconjunto enriquecido del DataFrame para el EDA...")
df_analisis = df_raw[columnas_enriquecidas].copy()

# 3. Limpieza de la variable objetivo (Eliminar valores nulos).
print(f"Tamaño original del subconjunto: {df_analisis.shape}")
df_analisis = df_analisis[df_analisis['CVDINFR4'].isin([1.0, 2.0])]
print(f"Tamaño después de limpiar valores faltantes en el target: {df_analisis.shape}")

# 4. Transformación de la variable objetivo (Mapeo a 0 y 1) - NUEVO PASO
print("Mapeando la variable objetivo a formato binario (1 para 'Sí', 0 para 'No')...")
df_analisis['CVDINFR4'] = df_analisis['CVDINFR4'].map({1.0: 1, 2.0: 0})
print("Mapeo completado.")

# 5. Genera el reporte con el DataFrame limpio, transformado y enriquecido.
print("Generando reporte con Sweetviz...")
try:
    reporte_enriquecido = sv.analyze(df_analisis, target_feat='CVDINFR4')
    reporte_enriquecido.show_notebook()
    print("✅ Reporte enriquecido generado exitosamente.")
except Exception as e:
    print(f"❌ Fallo al generar el reporte: {e}")

In [ ]:
# =============================================================================
# --- 6. GUARDAR DATAFRAME PROCESADO (NUEVO PASO FINAL) ---
# =============================================================================
# Objetivo: Guardar el DataFrame limpio para que otros notebooks puedan usarlo
# directamente sin repetir los pasos de limpieza.

print("Guardando el DataFrame procesado...")

# Crear la ruta al directorio de datos procesados
ruta_procesados = os.path.join('..', 'data', '02_processed')
# Asegurarse de que el directorio exista
os.makedirs(ruta_procesados, exist_ok=True)

# Guardar el DataFrame como un archivo CSV
ruta_archivo_limpio = os.path.join(ruta_procesados, 'brfss_2022_cleaned.csv')
df_analisis.to_csv(ruta_archivo_limpio, index=False)

print(f"✅ DataFrame guardado exitosamente en: '{ruta_archivo_limpio}'")